# Continue with the transformation process
In this notebook we will make functions to transform the data into a prepared dataset to be processed by the model.

In [88]:
# Libraries used:
import pandas as pd
import numpy as np
import ast
import os
import warnings
from pyod.models.knn import KNN
import pickle

# config 
pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')

The first step is to unify the data obtained and the corrupted data (if any).

In [89]:
def load_data():
    ''' 
    This function unify the raw data extrated with a corrupted if it exist any

    Returns:
    --------
    df: DataFrame with the raw data unified.
    '''
    # comprobate if the file exist
    if os.path.exists('../data/raw/data_raw.csv'):
        df = pd.read_csv('../data/raw/data_raw.csv')
    else:
        print('The file raw_data.csv does not exist')
    
    if os.path.exists('../data/raw/data_raw_corrupted.csv'):
        df_corrupted = pd.read_csv('../data/raw/data_raw_corrupted.csv')
        df = pd.concat([df, df_corrupted])
    else:
        print('There is no corrupted files')

    df.drop_duplicates(inplace=True)
    df.reset_index(drop=True, inplace=True)
    
    return df

In [90]:
df = load_data()
df

,propertyCode,thumbnail,numPhotos,price,propertyType,operation,size,exterior,rooms,bathrooms,address,province,municipality,country,latitude,longitude,showAddress,url,description,hasVideo,status,newDevelopment,priceByArea,detailedType,suggestedTexts,hasPlan,has3DTour,has360,hasStaging,topNewDevelopment,superTopHighlight,externalReference,floor,district,hasLift,parkingSpace,neighborhood,labels,newDevelopmentFinished
0,97942302,https://img3.idealista.com/blur/WEB_LISTING/0/...,23,850.0,countryHouse,rent,140.0,False,5,2,Riba-Roja de Túria,València,Riba-Roja de Túria,es,39.544071,-0.566219,False,https://www.idealista.com/inmueble/97942302/,254-HORTA23- Bonita y luminosa vivienda en Rib...,False,good,False,6.0,"{'typology': 'countryHouse', 'subTypology': 'c...","{'subtitle': 'Riba-Roja de Túria', 'title': 'C...",False,False,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,97956954,https://img3.idealista.com/blur/WEB_LISTING/0/...,18,900.0,duplex,rent,100.0,True,3,3,Calle peñiscola,València,Manises,es,39.499602,-0.474603,False,https://www.idealista.com/inmueble/97956954/,Luminoso piso duplex de 3 habitaciones en mani...,False,good,False,9.0,"{'typology': 'flat', 'subTypology': 'duplex'}","{'subtitle': 'Alameda Park, Manises', 'title':...",False,False,False,False,False,False,47,1,Alameda Park,True,"{'hasParkingSpace': True, 'isParkingSpaceInclu...",NaN,NaN,NaN
2,98200619,https://img3.idealista.com/blur/WEB_LISTING/0/...,26,1200.0,flat,rent,145.0,True,3,2,Calle de Conca,València,València,es,39.466220,-0.387735,False,https://www.idealista.com/inmueble/98200619/,GRAN VÍA INMOBILIARIA alquila piso en exclusiv...,False,good,False,8.0,{'typology': 'flat'},"{'subtitle': 'Arrancapins, València', 'title':...",False,False,False,False,False,False,A587E,2,Extramurs,True,NaN,Arrancapins,NaN,NaN
3,98341195,https://img3.idealista.com/blur/WEB_LISTING/0/...,48,2700.0,chalet,rent,339.0,False,4,5,Avenida Buganvillas,València,Godella,es,39.529005,-0.425450,False,https://www.idealista.com/inmueble/98341195/,Lion Capital Group le ofrece este espectacular...,True,good,False,8.0,"{'typology': 'chalet', 'subTypology': 'terrace...","{'subtitle': 'Campolivar, Godella', 'title': '...",True,False,False,False,False,False,LCGV-155,NaN,Campolivar,NaN,"{'hasParkingSpace': True, 'isParkingSpaceInclu...",NaN,NaN,NaN
4,98015046,https://img3.idealista.com/blur/WEB_LISTING/0/...,21,1800.0,flat,rent,103.0,True,3,2,Calle del Cura Planelles,València,València,es,39.470266,-0.331696,False,https://www.idealista.com/inmueble/98015046/,Lion Capital Group le ofrece este precioso pis...,True,good,False,17.0,{'typology': 'flat'},"{'subtitle': 'El Cabanyal-El Canyamelar, Valèn...",False,False,False,False,False,False,LCGV-136,bj,Poblats Marítims,True,NaN,El Cabanyal-El Canyamelar,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3017,98214599,https://img3.idealista.com/blur/WEB_LISTING/0/...,34,995.0,flat,rent,104.0,False,3,2,Plaça Num 137 Res Urb,València,València,es,39.441000,-0.381710,False,https://www.idealista.com/obra-nueva/98214599/,NaN,False,newdevelopment,True,10.0,{'typology': 'flat'},"{'subtitle': 'Malilla, València', 'title': 'Pi...",True,False,False,False,False,False,a1Q3W0000008s2sUAA,2,Quatre Carreres,True,NaN,Malilla,NaN,False
3018,98214582,https://img3.idealista.com/blur/WEB_LISTING/0/...,34,1085.0,flat,rent,109.0,False,3,2,Plaça Num 137 Res Urb,València,València,es,39.441400,-0.385010,False,https://www.idealista.com/obra-nueva/98214582/,NaN,False,newdevelopment,True,10.0,{'typology': 'flat'},"{'subtitle': 'Malilla, València', 'title': 'Pi...",True,False,False,False,False,False,a1Q3W0000008s0ZUAQ,1,Quatre Carreres,True,NaN,Malilla,NaN,False
3019,98091690,https://img3.idealista.com/blur/WEB_LISTING/0/...,34,1045.0,flat,rent,103.0,False,3,2,Plaça Num 137 Res Urb,València,València,es,39.439600,-0.382210,False,https://www.idealista.com/obra-nueva

Filter by the columns we need to work with:

In [91]:
cols_needed = [
    'propertyCode', 
    'price', 
    'numPhotos', 
    'size',
    'floor',
    'rooms', 'bathrooms', 
    'latitude', 'longitude', 
    'propertyType',
    'status',
    'parkingSpace', 
    'exterior',
    'hasLift', 'hasPlan', 'has360', 'has3DTour', 'hasVideo',
    'newDevelopmentFinished',  
]

# drop the columns that are not needed
df = df[cols_needed]
df

,propertyCode,price,numPhotos,size,floor,rooms,bathrooms,latitude,longitude,propertyType,status,parkingSpace,exterior,hasLift,hasPlan,has360,has3DTour,hasVideo,newDevelopmentFinished
0,97942302,850.0,23,140.0,NaN,5,2,39.544071,-0.566219,countryHouse,good,NaN,False,NaN,False,False,False,False,NaN
1,97956954,900.0,18,100.0,1,3,3,39.499602,-0.474603,duplex,good,"{'hasParkingSpace': True, 'isParkingSpaceInclu...",True,True,False,False,False,False,NaN
2,98200619,1200.0,26,145.0,2,3,2,39.466220,-0.387735,flat,good,NaN,True,True,False,False,False,False,NaN
3,98341195,2700.0,48,339.0,NaN,4,5,39.529005,-0.425450,chalet,good,"{'hasParkingSpace': True, 'isParkingSpaceInclu...",False,NaN,True,False,False,True,NaN
4,98015046,1800.0,21,103.0,bj,3,2,39.470266,-0.331696,flat,good,NaN,True,True,False,False,False,True,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3017,98214599,995.0,34,104.0,2,3,2,39.441000,-0.381710,flat,newdevelopment,NaN,False,True,True,False,False,False,False
3018,98214582,1085.0,34,109.0,1,3,2,39.441400,-0.385010,flat,newdevelopment,NaN,False,True,True,False,False,False,False
3019,98091690,1045.0,34,103.0,10,3,2,39.439600,-0.382210,flat,newdevelopment,NaN,False,True,True,False,False,False,False
3020,98091689,1040.0,34,83.0,10,2,2,39.437800,-0.384510,flat,newdevelopment,NaN,False,True,True,False,False,False,False


In [92]:
# Make propertyCode a index
df.set_index('propertyCode', inplace=True)

In [93]:
# See nan values
df.isnull().sum().sort_values(ascending=False)

newDevelopmentFinished    2987
parkingSpace              2153
floor                      371
hasLift                    335
hasVideo                     0
has3DTour                    0
has360                       0
hasPlan                      0
exterior                     0
price                        0
numPhotos                    0
propertyType                 0
longitude                    0
latitude                     0
bathrooms                    0
rooms                        0
size                         0
status                       0
dtype: int64

The nexts steps will we to create a functions that process all nan values and bad formatted data.

In [94]:
def process_status(df):
    '''
    This function process the status of the property.

    Parameters:
    -----------
    df: DataFrame with the raw data of status, newDevelopmentFinished.

    Returns:
    --------
    df: DataFrame with the raw data processed.
    '''

    # Create the columns that will replace the status column: renew and new_development.
    df['renew'] = df['status'].apply(lambda x: True if x == 'renew' else False)
    df['new_development'] = df['status'].apply(lambda x: True if x == 'newdevelopment' else False)

    # drop the status column
    df.drop('status', axis=1, inplace=True)

    # Same with newDevelopmentFinished
    df['isFinished'] = df['newDevelopmentFinished'].apply(lambda x: False if x == False else True)
    df.drop('newDevelopmentFinished', axis=1, inplace=True)

    return df

In [95]:
df = process_status(df)
df

,price,numPhotos,size,floor,rooms,bathrooms,latitude,longitude,propertyType,parkingSpace,exterior,hasLift,hasPlan,has360,has3DTour,hasVideo,renew,new_development,isFinished
propertyCode,,,,,,,,,,,,,,,,,,,
97942302,850.0,23,140.0,NaN,5,2,39.544071,-0.566219,countryHouse,NaN,False,NaN,False,False,False,False,False,False,True
97956954,900.0,18,100.0,1,3,3,39.499602,-0.474603,duplex,"{'hasParkingSpace': True, 'isParkingSpaceInclu...",True,True,False,False,False,False,False,False,True
98200619,1200.0,26,145.0,2,3,2,39.466220,-0.387735,flat,NaN,True,True,False,False,False,False,False,False,True
98341195,2700.0,48,339.0,NaN,4,5,39.529005,-0.425450,chalet,"{'hasParkingSpace': True, 'isParkingSpaceInclu...",False,NaN,True,False,False,True,False,False,True
98015046,1800.0,21,103.0,bj,3,2,39.470266,-0.331696,flat,NaN,True,True,False,False,False,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98214599,995.0,34,104.0,2,3,2,39.441000,-0.381710,flat,NaN,False,True,True,False,False,False,False,True,False
98214582,1085.0,34,109.0,1,3,2,39.441400,-0.385010,flat,NaN,False,True,True,False,False,False,False,True,False
98091690,1045.0,34,103.0,10,3,2,39.439600,-0.382210,flat,NaN,False,True,True,False,False,False,False,True,False


Same with parkingSpace

In [96]:
def process_parkingSpace(df):
    '''
    This function process the bad formatted parkingSpace

    Parameters:
    -----------
    df: DataFrame with the raw data of parkingSpace

    Returns:
    --------
    df: DataFrame with the raw data processed.
    '''
    # replace all the ' to " in the parkingSpace column
    df['parkingSpace'] = df['parkingSpace'].str.replace('\'', '"')

    # convert the string to a dictionary
    df['parkingSpace'] = df['parkingSpace'].apply(
        lambda x: ast.literal_eval(x) 
        if type(x) == str else x
        )

    # get the hasParkingSpace of the dict of the parkingSpace column
    df['hasParkingSpace'] = df['parkingSpace'].apply(
        lambda x: x['hasParkingSpace'] if type(x) == dict else False
        )

    # get isParkingSpaceIncludedInPrice 
    df['isParkingSpaceIncludedInPrice'] = df['parkingSpace'].apply(
        lambda x: x['isParkingSpaceIncludedInPrice'] if type(x) == dict else False
        )

    # get the parkingSpacePrice 
    df['parkingSpacePrice'] = df['parkingSpace'].apply(
        lambda x: x['parkingSpacePrice'] 
        if type(x) == dict and 'parkingSpacePrice' in x else 0
        )

    # drop the parkingSpace column
    df.drop(columns=['parkingSpace'], inplace=True)
    return df

In [97]:
df = process_parkingSpace(df)
df

,price,numPhotos,size,floor,rooms,bathrooms,latitude,longitude,propertyType,exterior,hasLift,hasPlan,has360,has3DTour,hasVideo,renew,new_development,isFinished,hasParkingSpace,isParkingSpaceIncludedInPrice,parkingSpacePrice
propertyCode,,,,,,,,,,,,,,,,,,,,,
97942302,850.0,23,140.0,NaN,5,2,39.544071,-0.566219,countryHouse,False,NaN,False,False,False,False,False,False,True,False,False,0.0
97956954,900.0,18,100.0,1,3,3,39.499602,-0.474603,duplex,True,True,False,False,False,False,False,False,True,True,True,0.0
98200619,1200.0,26,145.0,2,3,2,39.466220,-0.387735,flat,True,True,False,False,False,False,False,False,True,False,False,0.0
98341195,2700.0,48,339.0,NaN,4,5,39.529005,-0.425450,chalet,False,NaN,True,False,False,True,False,False,True,True,True,0.0
98015046,1800.0,21,103.0,bj,3,2,39.470266,-0.331696,flat,True,True,False,False,False,True,False,False,True,False,False,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98214599,995.0,34,104.0,2,3,2,39.441000,-0.381710,flat,False,True,True,False,False,False,False,True,False,False,False,0.0
98214582,1085.0,34,109.0,1,3,2,39.441400,-0.385010,flat,False,True,True,False,False,False,False,True,False,False,False,0.0
98091690,1045.0,34,103.0,10,3,2,39.439600,-0.382210,flat,False,True,True,False,False,False,False,True,False,False,False,0.0


In [98]:
def process_floor(df):
    ''' 
    This function process the bad formatted floor

    Parameters:
    -----------
    df: DataFrame with the raw data of floor

    Returns:
    --------
    df: DataFrame with the raw data processed.
    '''
    floor_nan = df[df['floor'].isnull()] # get the nan values
    property_type = floor_nan['propertyType'].value_counts().index.to_list()

    for i in property_type:
        # get the mode of the propertyType column
        mode = df['floor'][df['propertyType'] == i].mode()[0]
        # get the index of the rows with the propertyType i and the floor is null
        index = df[(df['propertyType'] == i) & (df['floor'].isnull())].index
        # replace the nan with the mode
        df.loc[index, 'floor'] = mode

    # where floor is 'bj' or 'en' put 0
    df.loc[df['floor'] == 'bj', 'floor'] = 0
    df.loc[df['floor'] == 'en', 'floor'] = 0

    # where floor is 'st' put 1
    df.loc[df['floor'] == 'st', 'floor'] = -1

    return df

In [99]:
df = process_floor(df)
df

,price,numPhotos,size,floor,rooms,bathrooms,latitude,longitude,propertyType,exterior,hasLift,hasPlan,has360,has3DTour,hasVideo,renew,new_development,isFinished,hasParkingSpace,isParkingSpaceIncludedInPrice,parkingSpacePrice
propertyCode,,,,,,,,,,,,,,,,,,,,,
97942302,850.0,23,140.0,0,5,2,39.544071,-0.566219,countryHouse,False,NaN,False,False,False,False,False,False,True,False,False,0.0
97956954,900.0,18,100.0,1,3,3,39.499602,-0.474603,duplex,True,True,False,False,False,False,False,False,True,True,True,0.0
98200619,1200.0,26,145.0,2,3,2,39.466220,-0.387735,flat,True,True,False,False,False,False,False,False,True,False,False,0.0
98341195,2700.0,48,339.0,0,4,5,39.529005,-0.425450,chalet,False,NaN,True,False,False,True,False,False,True,True,True,0.0
98015046,1800.0,21,103.0,0,3,2,39.470266,-0.331696,flat,True,True,False,False,False,True,False,False,True,False,False,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98214599,995.0,34,104.0,2,3,2,39.441000,-0.381710,flat,False,True,True,False,False,False,False,True,False,False,False,0.0
98214582,1085.0,34,109.0,1,3,2,39.441400,-0.385010,flat,False,True,True,False,False,False,False,True,False,False,False,0.0
98091690,1045.0,34,103.0,10,3,2,39.439600,-0.382210,flat,False,True,True,False,False,False,False,True,False,False,False,0.0


In [100]:
def process_hasLift(df):
    ''' 
    This function process all nan values of hasLift

    Parameters:
    -----------
    df: DataFrame with the raw data of hasLift

    Returns:
    --------
    df: DataFrame with the raw data processed.
    '''
    # get the index of the rows with the hasLift is null
    index = df[df['hasLift'].isnull()].index
    # replace the nan with False
    df.loc[index, 'hasLift'] = False
    
    return df

In [101]:
df = process_hasLift(df) 
df.isnull().sum().sort_values(ascending=False)

price                            0
hasPlan                          0
isParkingSpaceIncludedInPrice    0
hasParkingSpace                  0
isFinished                       0
new_development                  0
renew                            0
hasVideo                         0
has3DTour                        0
has360                           0
hasLift                          0
numPhotos                        0
exterior                         0
propertyType                     0
longitude                        0
latitude                         0
bathrooms                        0
rooms                            0
floor                            0
size                             0
parkingSpacePrice                0
dtype: int64

Let's force-secure the type of the data.

In [102]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3022 entries, 97942302 to 91245678
Data columns (total 21 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   price                          3022 non-null   float64
 1   numPhotos                      3022 non-null   int64  
 2   size                           3022 non-null   float64
 3   floor                          3022 non-null   object 
 4   rooms                          3022 non-null   int64  
 5   bathrooms                      3022 non-null   int64  
 6   latitude                       3022 non-null   float64
 7   longitude                      3022 non-null   float64
 8   propertyType                   3022 non-null   object 
 9   exterior                       3022 non-null   bool   
 10  hasLift                        3022 non-null   object 
 11  hasPlan                        3022 non-null   bool   
 12  has360                         3022 n

In [103]:
def retype_data(df):
    ''' 
    This function forcetype all columns of the dataset

    Parameters:
    -----------
    df: DataFrame with the raw data

    Returns:
    --------
    df: DataFrame with secure types.
    '''

    # int types
    df['numPhotos'] = df['numPhotos'].astype(int)
    df['floor'] = df['floor'].astype(int)
    df['rooms'] = df['rooms'].astype(int)
    df['bathrooms'] = df['bathrooms'].astype(int)

    # float types
    df['price'] = df['price'].astype(float)
    df['size'] = df['size'].astype(float)
    df['parkingSpacePrice'] = df['parkingSpacePrice'].astype(float)
    df['latitude'] = df['latitude'].astype(float)
    df['longitude'] = df['longitude'].astype(float)

    # boolean types
    df['exterior'] = df['exterior'].astype(bool)
    df['renew'] = df['renew'].astype(bool)
    df['new_development'] = df['new_development'].astype(bool)
    df['hasParkingSpace'] = df['hasParkingSpace'].astype(bool)
    df['isParkingSpaceIncludedInPrice'] = df['isParkingSpaceIncludedInPrice'].astype(bool)
    df['isFinished'] = df['isFinished'].astype(bool)
    df['hasLift'] = df['hasLift'].astype(bool)
    df['hasPlan'] = df['hasPlan'].astype(bool)
    df['has360'] = df['has360'].astype(bool)
    df['has3DTour'] = df['has3DTour'].astype(bool)
    df['hasVideo'] = df['hasVideo'].astype(bool)

    # object types
    df['propertyType'] = df['propertyType'].astype(str)

    order_of_cols = ['price', 'numPhotos', 'floor', 'rooms', 'bathrooms',
    'size', 'parkingSpacePrice', 'latitude', 'longitude', 'exterior', 'renew',
    'new_development', 'hasParkingSpace', 'isParkingSpaceIncludedInPrice',
    'isFinished', 'hasLift', 'hasPlan', 'has360', 'has3DTour', 'hasVideo',
    'propertyType']
    
    df = df[order_of_cols]
    return df

In [104]:
df = retype_data(df)
df.shape

(3022, 21)

Nows let's deal with the outliers.

In [105]:
def drop_outliers(df):
    '''
    This function drop the outliers of the dataset by KNN method

    Parameters:
    -----------
    df: DataFrame with the outliers

    Returns:
    --------
    df: DataFrame with the outliers dropped.
    '''
    num_cols = df.select_dtypes(include=['int64', 'float64']).columns
    
    clf = KNN()
    clf.fit(df[num_cols])
    y_pred = clf.predict(df[num_cols])

    print(f'The percentage of outliers is {100*sum(y_pred)/len(y_pred)}%')
    
    df = df[y_pred == 0]
    return df


In [106]:
df = drop_outliers(df) 
df.shape

The percentage of outliers is 8.669755129053607%


(2760, 21)

At this point we automated the process of data cleaning, let's automate the transformation of address.

In [107]:
def clustering_address(df):
    '''
    This function create cluster from the latitude and longitude a append it to the dataset

    Parameters:
    -----------
    df: DataFrame with the latitude and longitude

    Returns:
    --------
    df: DataFrame with the latitude and longitude clustered.
    '''
    # load the clustering model
    cluster = pickle.load(open('../models/kmeans_clustering.pkl', 'rb'))
    # get the cluster of the latitude and longitude
    df['cluster'] = cluster.predict(df[['latitude', 'longitude']])

    # drop the latitude and longitude columns
    df.drop(columns=['latitude', 'longitude'], inplace=True)

    return df

In [108]:
df = clustering_address(df)
df.cluster.value_counts()

0    2250
2     197
1     177
3     136
Name: cluster, dtype: int64

In [109]:
df.sample(5)

,price,numPhotos,floor,rooms,bathrooms,size,parkingSpacePrice,exterior,renew,new_development,hasParkingSpace,isParkingSpaceIncludedInPrice,isFinished,hasLift,hasPlan,has360,has3DTour,hasVideo,propertyType,cluster
propertyCode,,,,,,,,,,,,,,,,,,,,
98254835,3500.0,25,0,5,3,350.0,0.0,False,False,False,True,True,True,False,False,False,False,False,chalet,3
97525641,2150.0,33,2,2,2,99.0,0.0,True,False,False,False,False,True,True,True,False,False,False,flat,0
97019989,950.0,31,1,3,1,75.0,0.0,True,False,False,False,False,True,True,False,False,False,False,flat,0
32771983,950.0,7,3,4,1,84.0,0.0,True,False,False,False,False,True,True,False,False,False,False,flat,0
36291521,620.0,10,7,0,1,40.0,0.0,True,False,False,True,True,True,True,False,False,False,False,studio,0


This looks like a good dataset to do feature engineering.